# Transfer Learning CIFAR10

* Train a simple convnet on the CIFAR dataset the first 5 output classes [0..4].
* Freeze convolutional layers and fine-tune dense layers for the last 5 ouput classes [5..9].


### 1. Import CIFAR10 data and create 2 datasets with one dataset having classes from 0 to 4 and other having classes from 5 to 9 

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import keras
from keras.datasets import cifar10
from keras.layers import Dense, Convolution2D, Flatten, Activation, MaxPooling2D, Dropout
from keras.models import Sequential
from keras.utils import np_utils


Using TensorFlow backend.


In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()


In [3]:
#Shape info
print(x_train.shape,x_test.shape,y_train.shape,y_test.shape)
#class break down
print(np.unique(y_train, return_counts=True))
#Sample data
y_train[0:10]
np.isin(y_train[0:10],[0,1,2,3,4])


#labels = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
#labels1 = ['airplane', 'automobile', 'bird', 'cat', 'deer']
#labels2 = ['dog', 'frog', 'horse', 'ship', 'truck']
#target_label = labels.index(label)


(50000, 32, 32, 3) (10000, 32, 32, 3) (50000, 1) (10000, 1)
(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=uint8), array([5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000],
      dtype=int64))


array([[6],
       [9],
       [9],
       [4],
       [1],
       [1],
       [2],
       [7],
       [8],
       [3]], dtype=uint8)

array([[False],
       [False],
       [False],
       [ True],
       [ True],
       [ True],
       [ True],
       [False],
       [False],
       [ True]])

In [4]:
#len(x_train[y_train == target_label])
x1_train=x_train[np.isin(y_train,[0,1,2,3,4])]
y1_train=y_train[np.isin(y_train,[0,1,2,3,4])]
x1_test=x_test[np.isin(y_test,[0,1,2,3,4])]
y1_test=y_test[np.isin(y_test,[0,1,2,3,4])]

x2_train=x_train[np.isin(y_train,[5,6,7,8,9])]
y2_train=y_train[np.isin(y_train,[5,6,7,8,9])]
x2_test=x_test[np.isin(y_test,[5,6,7,8,9])]
y2_test=y_test[np.isin(y_test,[5,6,7,8,9])]




IndexError: boolean index did not match indexed array along dimension 1; dimension is 32 but corresponding boolean dimension is 1

In [5]:
y_train[0:5].isin([0,1,2,3,4])

AttributeError: 'numpy.ndarray' object has no attribute 'isin'

In [6]:
X1_train = []
X1_test = []
X2_train = []
X2_test = []
Y1_train = []
Y1_test = []
Y2_train = []
Y2_test = []

for ix in range(y_train.shape[0]):
    if y_train[ix] < 5:
        # put data in set 1
        X1_train.append(x_train[ix]/255.0)
        Y1_train.append(y_train[ix])
    else:
        # put data in set 2
        X2_train.append(x_train[ix]/255.0)
        Y2_train.append(y_train[ix])

for ix in range(y_test.shape[0]):
    if y_test[ix] < 5:
        # put data in set 1
        X1_test.append(x_test[ix]/255.0)
        Y1_test.append(y_test[ix])
    else:
        # put data in set 2
        X2_test.append(x_test[ix]/255.0)
        Y2_test.append(y_test[ix])

In [7]:
X1_train = np.asarray(X1_train).reshape((-1, 32, 32, 3))
X1_test = np.asarray(X1_test).reshape((-1, 32, 32, 3))
X2_train = np.asarray(X2_train).reshape((-1, 32, 32, 3))
X2_test = np.asarray(X2_test).reshape((-1, 32, 32, 3))

print(X1_train.shape, X1_test.shape)

(25000, 32, 32, 3) (5000, 32, 32, 3)


### 2. Use One-hot encoding to divide y_train and y_test into required no of output classes

In [8]:
Y1_train = np_utils.to_categorical(np.asarray(Y1_train), num_classes=5)
Y1_test = np_utils.to_categorical(np.asarray(Y1_test), num_classes=5)

Y2_train = np_utils.to_categorical(np.asarray(Y2_train), num_classes=10)
Y2_test = np_utils.to_categorical(np.asarray(Y2_test), num_classes=10)
print(Y1_train.shape, Y1_test.shape)

(25000, 5) (5000, 5)


In [9]:
split1 = int(0.8 * X1_train.shape[0])
split2 = int(0.8 * X2_train.shape[0])

x1_val = X1_train[split1:]
x1_train = X1_train[:split1]
y1_val = Y1_train[split1:]
y1_train = Y1_train[:split1]

x2_val = X2_train[split2:]
x2_train = X2_train[:split2]
y2_val = Y2_train[split2:]
y2_train = Y2_train[:split2]

### 3. Build a sequential neural network model which can classify the classes 0 to 4 of CIFAR10 dataset with at least 80% accuracy on test data

In [10]:
model = Sequential()

model.add(Convolution2D(32, 5, 5, input_shape=(32, 32, 3), activation='relu'))
model.add(Convolution2D(16, 5, 5, activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Convolution2D(8, 3, 3, activation='relu'))
model.add(Flatten())
model.add(Dropout(0.42))

model.add(Dense(128))
model.add(Activation('relu'))

model.add(Dense(5))
model.add(Activation('softmax'))

model.summary()
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 28, 28, 32)        2432      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 16)        12816     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 16)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 10, 10, 8)         1160      
_________________________________________________________________
flatten_1 (Flatten)          (None, 800)               0         
_________________________________________________________________
dropout_1 (Dropout)  

In [ ]:
import time, datetime

start = datetime.datetime.now()
hist1 = model.fit(x1_train, y1_train,
         nb_epoch=25,
         shuffle=True,
         batch_size=100,
         validation_data=(x1_val, y1_val), verbose=2)

time_taken = datetime.datetime.now() - start
print('\n'*2, '-'*20, '\n')
print('Time taken for first training: ', time_taken)
print('\n', '-'*20, '\n'*2)


Instructions for updating:
Use tf.cast instead.
Train on 20000 samples, validate on 5000 samples
Epoch 1/25
 - 44s - loss: 1.2176 - acc: 0.4768 - val_loss: 1.0249 - val_acc: 0.5832
Epoch 2/25
 - 45s - loss: 1.0298 - acc: 0.5749 - val_loss: 0.9874 - val_acc: 0.6094
Epoch 3/25
 - 43s - loss: 0.9533 - acc: 0.6142 - val_loss: 0.9027 - val_acc: 0.6424
Epoch 4/25
 - 43s - loss: 0.8903 - acc: 0.6374 - val_loss: 0.8492 - val_acc: 0.6704
Epoch 5/25
 - 43s - loss: 0.8552 - acc: 0.6571 - val_loss: 0.8276 - val_acc: 0.6774
Epoch 6/25
 - 43s - loss: 0.8074 - acc: 0.6803 - val_loss: 0.8320 - val_acc: 0.6740
Epoch 7/25
 - 43s - loss: 0.7842 - acc: 0.6948 - val_loss: 0.7970 - val_acc: 0.6902
Epoch 8/25
 - 43s - loss: 0.7533 - acc: 0.7101 - val_loss: 0.7615 - val_acc: 0.7030
Epoch 9/25
 - 43s - loss: 0.7232 - acc: 0.7196 - val_loss: 0.7796 - val_acc: 0.7038
Epoch 10/25
 - 44s - loss: 0.7009 - acc: 0.7265 - val_loss: 0.7570 - val_acc: 0.7220
Epoch 11/25
 - 45s - loss: 0.6783 - acc: 0.7392 - val_loss: 0.

### 4. In the model which was built above (for classification of classes 0-4 in CIFAR10), make only the dense layers to be trainable and conv layers to be non-trainable

In [0]:
for l in model.layers[:6]:
    l.trainable = False


### 5. Utilize the the model trained on CIFAR 10 (classes 0 to 4) to classify the classes 5 to 9 of CIFAR 10  (Use Transfer Learning) <br>
Achieve an accuracy of more than 85% on test data

In [0]:
trans_model = Sequential(model.layers[:6])

trans_model.add(Dense(128))
trans_model.add(Activation('relu'))
trans_model.add(Dense(10))
trans_model.add(Activation('softmax'))

trans_model.summary()
trans_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
start = datetime.datetime.now()
hist2 = trans_model.fit(x2_train, y2_train, nb_epoch=25, shuffle=True, batch_size=100, validation_data=(x2_val, y2_val), verbose=2)
time_taken = datetime.datetime.now() - start
print('\n'*2, '-'*20, '\n')
print('Time taken for first training: ', time_taken)
print('\n', '-'*20, '\n'*2)

# Text classification using TF-IDF

### 6. Load the dataset from sklearn.datasets

In [1]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

In [2]:
categories = ['alt.atheism', 'soc.religion.christian', 'comp.graphics', 'sci.med']

### 7. Training data

In [3]:
twenty_train = fetch_20newsgroups(subset='train', categories=categories, shuffle=True, random_state=42)

### 8. Test data

In [4]:
twenty_test = fetch_20newsgroups(subset='test', categories=categories, shuffle=True, random_state=42)

###  a.  You can access the values for the target variable using .target attribute 
###  b. You can access the name of the class in the target variable with .target_names


In [5]:
twenty_train.target

array([1, 1, 3, ..., 2, 2, 2], dtype=int64)

In [6]:
twenty_train.target_names

['alt.atheism', 'comp.graphics', 'sci.med', 'soc.religion.christian']

In [7]:
twenty_train.data[0:5]

['From: sd345@city.ac.uk (Michael Collier)\nSubject: Converting images to HP LaserJet III?\nNntp-Posting-Host: hampton\nOrganization: The City University\nLines: 14\n\nDoes anyone know of a good way (standard PC application/PD utility) to\nconvert tif/img/tga files into LaserJet III format.  We would also like to\ndo the same, converting to HPGL (HP plotter) files.\n\nPlease email any response.\n\nIs this the correct group?\n\nThanks in advance.  Michael.\n-- \nMichael Collier (Programmer)                 The Computer Unit,\nEmail: M.P.Collier@uk.ac.city                The City University,\nTel: 071 477-8000 x3769                      London,\nFax: 071 477-8565                            EC1V 0HB.\n',
 "From: ani@ms.uky.edu (Aniruddha B. Deglurkar)\nSubject: help: Splitting a trimming region along a mesh \nOrganization: University Of Kentucky, Dept. of Math Sciences\nLines: 28\n\n\n\n\tHi,\n\n\tI have a problem, I hope some of the 'gurus' can help me solve.\n\n\tBackground of the probl

### 9.  Now with dependent and independent data available for both train and test datasets, using TfidfVectorizer fit and transform the training data and test data and get the tfidf features for both

In [8]:
twenty_train.data[0]

'From: sd345@city.ac.uk (Michael Collier)\nSubject: Converting images to HP LaserJet III?\nNntp-Posting-Host: hampton\nOrganization: The City University\nLines: 14\n\nDoes anyone know of a good way (standard PC application/PD utility) to\nconvert tif/img/tga files into LaserJet III format.  We would also like to\ndo the same, converting to HPGL (HP plotter) files.\n\nPlease email any response.\n\nIs this the correct group?\n\nThanks in advance.  Michael.\n-- \nMichael Collier (Programmer)                 The Computer Unit,\nEmail: M.P.Collier@uk.ac.city                The City University,\nTel: 071 477-8000 x3769                      London,\nFax: 071 477-8565                            EC1V 0HB.\n'

In [9]:
len(twenty_train)

6

In [10]:
len(twenty_train.data)

2257

In [11]:

vectorizer = TfidfVectorizer(decode_error='ignore', strip_accents='unicode', analyzer='word', stop_words='english', 
                             lowercase=True, min_df=0.01, max_df=0.97, max_features=1800)
train_data = vectorizer.fit_transform(twenty_train.data)
test_data = vectorizer.transform(twenty_test.data)


In [14]:
# summarize
print(train_data.shape, test_data.shape)
print(pd.DataFrame(vectorizer.idf_))
print(vectorizer.vocabulary_)


(2257, 1800) (1502, 1800)
             0
0     4.137267
1     4.915572
2     4.279583
3     4.714902
4     4.279583
5     5.225727
6     4.644697
7     5.058673
8     3.301700
9     4.627890
10    3.604241
11    3.604241
12    3.563179
13    3.551751
14    3.315063
15    3.459545
16    3.731802
17    3.787761
18    3.622368
19    5.390030
20    5.464138
21    4.961035
22    2.932275
23    4.714902
24    5.058673
25    5.166887
26    3.404115
27    5.138716
28    4.984565
29    3.672379
...        ...
1770  3.977303
1771  4.661792
1772  4.872087
1773  3.529278
1774  3.634638
1775  3.109107
1776  4.378429
1777  3.698354
1778  2.640016
1779  5.111317
1780  4.488128
1781  4.893593
1782  4.810212
1783  4.021754
1784  4.579100
1785  3.968645
1786  1.587344
1787  4.418170
1788  3.968645
1789  3.370377
1790  3.301700
1791  5.058673
1792  5.288248
1793  4.003736
1794  3.076788
1795  5.390030
1796  3.480488
1797  4.488128
1798  4.595100
1799  5.390030

[1800 rows x 1 columns]
{'city': 341, 'ac':

In [15]:
train_data_tfid_features = pd.DataFrame(train_data.toarray(), columns=vectorizer.get_feature_names())
train_data_tfid_features

,00,000,01,02,03,04,05,08,10,100,...,wrote,wwc,yeah,year,years,yeast,yes,york,young,yoyo
0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
1,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
2,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.0,0.0,0.000000,0.098693,0.000000,0.000000,0.000000,0.000000,0.0
3,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.101147,0.0,...,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
4,0.128840,0.0,0.133272,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
5,0.077157,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
6,0.000000,0.0,0.077432,0.000000,0.000000,0.000000,0.000000,0.000000,0.119478,0.0,...,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
7,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
8,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.073300,0.0,0.0,0.000000,0.000000,0.000000,0.077269,0.000000,0.000000,0.0
9,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0


In [16]:
test_data_tfid_features = pd.DataFrame(test_data.toarray(), columns=vectorizer.get_feature_names())
print(test_data_tfid_features)

       00       000        01        02   03       04        05        08  \
0     0.0  0.000000  0.000000  0.000000  0.0  0.00000  0.000000  0.000000   
1     0.0  0.000000  0.000000  0.000000  0.0  0.00000  0.000000  0.000000   
2     0.0  0.000000  0.000000  0.000000  0.0  0.00000  0.000000  0.000000   
3     0.0  0.000000  0.000000  0.000000  0.0  0.00000  0.000000  0.000000   
4     0.0  0.000000  0.000000  0.000000  0.0  0.00000  0.000000  0.000000   
5     0.0  0.000000  0.000000  0.000000  0.0  0.00000  0.000000  0.000000   
6     0.0  0.000000  0.000000  0.000000  0.0  0.00000  0.000000  0.000000   
7     0.0  0.000000  0.000000  0.127816  0.0  0.00000  0.000000  0.000000   
8     0.0  0.000000  0.000000  0.000000  0.0  0.00000  0.000000  0.000000   
9     0.0  0.000000  0.000000  0.000000  0.0  0.00000  0.000000  0.000000   
10    0.0  0.000000  0.000000  0.000000  0.0  0.00000  0.000000  0.000000   
11    0.0  0.000000  0.000000  0.000000  0.0  0.00000  0.000000  0.000000   

### 10. Use logisticRegression with tfidf features as input and targets as output and train the model and report the train and test accuracy score

In [17]:
train_data = train_data.todense()
test_data = test_data.todense()
train_labels = twenty_train.target
test_labels = twenty_test.target
print(train_data.shape,train_labels.shape, test_data.shape,test_labels.shape)

(2257, 1800) (2257,) (1502, 1800) (1502,)


In [18]:
model_logr = LogisticRegression()	#using logistic regression
model_logr.fit(train_data, train_labels)
test_pred = model_logr.predict(test_data)

In [19]:
print(model_logr.score(test_data, test_labels))	
print(metrics.classification_report(test_labels, test_pred))
print(metrics.confusion_matrix(test_labels, test_pred))

0.8761651131824234
             precision    recall  f1-score   support

          0       0.87      0.75      0.81       319
          1       0.86      0.95      0.91       389
          2       0.91      0.85      0.88       396
          3       0.86      0.93      0.89       398

avg / total       0.88      0.88      0.87      1502

[[240  12  19  48]
 [  6 370  12   1]
 [ 13  37 336  10]
 [ 16   9   3 370]]
